Cloning into 'llama_index'...
remote: Enumerating objects: 16817, done.
remote: Counting objects: 100% (3393/3393), done.
remote: Compressing objects: 100% (1316/1316), done.
remote: Total 16817 (delta 2628), reused 2268 (delta 2070), pack-reused 13424
Receiving objects: 100% (16817/16817), 45.34 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (11017/11017), done.


YT Ref -> https://www.youtube.com/watch?v=cNMYeW2mpBs

In [ ]:
!git clone https://github.com/jerryjliu/llama_index.git

In [ ]:
!pip install llama-index llama_hub wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.0/335.0 kB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
     

In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "COPY AND PASTE YOUR OPENAI API HERE"
openai.api_key = os.environ["OPENAI_API_KEY"]

# Data connectors (LlamaHub)

In [ ]:
from llama_hub.wikipedia.base import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin', 'Rome', 'Tokyo', 'Canberra', 'Santiago'])

# Basic query functionalities

In [ ]:
from llama_index import VectorStoreIndex
# build an index over these Document objects.
index = VectorStoreIndex.from_documents(documents)
# you can query an index with the default QueryEngine
query_engine = index.as_query_engine()
response = query_engine.query("How many people live in Berlin")

In [ ]:
print(response)


According to the context information, Berlin has a population of approximately 3.7 million inhabitants.


# Query Multiple Documents:
Source: https://gpt-index.readthedocs.io/en/latest/examples/usecases/10q_sub_question.html

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, VectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [ ]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [ ]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 7.7 MB/s eta 0:00:00


In [ ]:
ls llama_index/docs/examples/data/10q/uber_10q_sept_2022.pdf

llama_index/docs/examples/data/10q/uber_10q_sept_2022.pdf


In [ ]:
march_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_march_2022.pdf"]).load_data()
june_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_june_2022.pdf"]).load_data()
sept_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_sept_2022.pdf"]).load_data()


In [ ]:
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)

In [ ]:
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(name='sept_22', description='Provides information about Uber quarterly financials ending September 2022')
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(name='june_22', description='Provides information about Uber quarterly financials ending June 2022')
    ),
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(name='march_22', description='Provides information about Uber quarterly financials ending March 2022')
    ),
]

In [ ]:
# Given a query, this query engine `SubQuestionQueryEngine ` will generate a “query plan”
# containing sub-queries against sub-documents before synthesizing the final answer.
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
response = s_engine.query('Analyze Uber revenue growth over the latest two quarter filings')


Generated 2 sub questions.
[sept_22] Q: What is the revenue growth of Uber for the quarter ending September 2022
[sept_22] A: compared to the same period in 2021?

The revenue growth of Uber for the quarter ending September 2022 compared to the same period in 2021 is 72%.
[june_22] Q: What is the revenue growth of Uber for the quarter ending June 2022
[june_22] A: compared to the same period in 2021?

Revenue increased $4.1 billion, or 105%, for the three months ended June 30, 2022 compared to the same period in 2021.


In [ ]:
print(response)


Uber's revenue growth over the latest two quarter filings has been strong, with a 72% increase for the quarter ending September 2022 compared to the same period in 2021, and a 105% increase for the quarter ending June 2022 compared to the same period in 2021.


# Router

define a custom router query engine that can route to either a SQL database or a vector database.

Source: https://gpt-index.readthedocs.io/en/latest/examples/query_engine/SQLRouterQueryEngine.html

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SQLStructStoreIndex,
    SQLDatabase,
    WikipediaReader
)

In [ ]:
# Create Database Schema + Test Data
# Here we introduce a toy scenario where there are 100 tables (too big to fit into the prompt)

from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)
# print tables
metadata_obj.tables.keys()


dict_keys(['city_stats'])

In [ ]:
from sqlalchemy import insert
rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [ ]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


In [ ]:
# Load Data
# We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.
cities = ['Toronto', 'Berlin', 'Tokyo']
wiki_docs = WikipediaReader().load_data(pages=cities)

In [ ]:
# Build SQL Index
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
sql_index = SQLStructStoreIndex.from_documents(
    [],
    sql_database=sql_database,
    table_name="city_stats",
)

In [ ]:
# Build Vector Index
# build a separate vector index per city
# You could also choose to define a single vector index across all docs, and annotate each chunk by metadata
vector_indices = []
for wiki_doc in wiki_docs:
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    vector_indices.append(vector_index)

In [ ]:
# Define Query Engines, Set as Tools
sql_query_engine = sql_index.as_query_engine()
vector_query_engines = [index.as_query_engine() for index in vector_indices]
from llama_index.tools.query_engine import QueryEngineTool


sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        'Useful for translating a natural language query into a SQL query over a table containing: '
        'city_stats, containing the population/country of each city'
    )
)
vector_tools = []
for city, query_engine in zip(cities, vector_query_engines):
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f'Useful for answering semantic questions about {city}',
    )
    vector_tools.append(vector_tool)

In [ ]:
# Define Router Query Engine
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + vector_tools)
)
response = query_engine.query('Which city has the highest population?')

In [ ]:
response

Response(response=' Tokyo has the highest population, with 13,960,000 people.', source_nodes=[], extra_info={'result': [('Tokyo', 13960000)], 'sql_query': 'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;'})

In [ ]:
print(str(response))

 Tokyo has the highest population, with 13,960,000 people.


In [ ]:
response = query_engine.query('Tell me about the historical museums in Berlin')


In [ ]:
response

Response(response="\nBerlin is home to many historical museums, including the Museum Island on the River Spree, which houses five museums built between 1830 and 1930 and is a UNESCO World Heritage site. The Altes Museum, Neues Museum, Alte Nationalgalerie, Pergamon Museum, and Bode Museum are all located on the Museum Island. The German Museum of Technology in Kreuzberg has a large collection of historical technical artifacts, and the Museum für Naturkunde (Berlin's natural history museum) exhibits natural history near Berlin Hauptbahnhof. The Jewish Museum has a standing exhibition on two millennia of German-Jewish history, and the Beate Uhse Erotic Museum was once the largest erotic museum in the world. The Brücke Museum features one of the largest collections of works by artists of the early 20th-century expressionist movement, and the Stasi Museum is located on the grounds of the former East German Ministry for State Security. The site of Checkpoint Charlie, one of the most renowne

In [ ]:
print(str(response))


Berlin is home to many historical museums, including the Museum Island on the River Spree, which houses five museums built between 1830 and 1930 and is a UNESCO World Heritage site. The Altes Museum, Neues Museum, Alte Nationalgalerie, Pergamon Museum, and Bode Museum are all located on the Museum Island. The German Museum of Technology in Kreuzberg has a large collection of historical technical artifacts, and the Museum für Naturkunde (Berlin's natural history museum) exhibits natural history near Berlin Hauptbahnhof. The Jewish Museum has a standing exhibition on two millennia of German-Jewish history, and the Beate Uhse Erotic Museum was once the largest erotic museum in the world. The Brücke Museum features one of the largest collections of works by artists of the early 20th-century expressionist movement, and the Stasi Museum is located on the grounds of the former East German Ministry for State Security. The site of Checkpoint Charlie, one of the most renowned crossing points of

# Hypothetical document embeddings (HyDE)

Source: https://gpt-index.readthedocs.io/en/latest/examples/query_transformations/HyDEQueryTransformDemo.html

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from IPython.display import Markdown, display


In [ ]:
# load documents
documents = SimpleDirectoryReader('llama_index/examples/paul_graham_essay/data').load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_str = "what did paul graham do after going to RISD"

In [ ]:
#Now, we use HyDEQueryTransform to generate a hypothetical document and use it for embedding lookup.
hyde = HyDEQueryTransform(include_original=True)
query_engine = index.as_query_engine()
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>
After going to RISD, Paul Graham did freelance work for a group that did projects for customers. He also moved to Providence and then to New York City, where he became a New York artist. He looked for an apartment to buy and then moved to Cambridge, Massachusetts, where he started working on a new dialect of Lisp called Arc.</b>

In [ ]:
#In this example, HyDE improves output quality significantly, by hallucinating accurately what Paul Graham did after RISD (see below), and thus improving the embedding quality, and final output.
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [ ]:
hyde_doc

'\nAfter graduating from the Rhode Island School of Design (RISD) in 1985, Paul Graham went on to pursue a career in computer programming. He worked as a software developer for several companies, including Viaweb, which he co-founded in 1995. Viaweb was eventually acquired by Yahoo in 1998, and Graham used the proceeds to become a venture capitalist. He founded Y Combinator in 2005, a startup accelerator that has helped launch over 2,000 companies, including Dropbox, Airbnb, and Reddit. Graham has also written several books on programming and startups, and he continues to be an active investor in the tech industry.'

# Use LlamaIndex with LangChain

Source: https://github.com/jerryjliu/llama_index/blob/main/examples/langchain_demo/LangchainDemo.ipynb

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
# Using LlamaIndex as a Callable Tool
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('llama_index/examples/paul_graham_essay/data').load_data()
index = VectorStoreIndex.from_documents(documents=documents)
tools = [
    Tool(
        name = "LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)
agent_executor.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [ ]:
agent_executor.run(input="What did the author do growing up?")

'\nThe author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-stabilized apartment, building an online store builder, editing Lisp expressions, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Using LlamaIndex as a memory module

from langchain import OpenAI
from langchain.llms import OpenAIChat
from langchain.agents import initialize_agent

from llama_index import ListIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory
index = ListIndex([])
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

In [ ]:
memory = GPTIndexChatMemory(
    index=index,
    memory_key="chat_history",
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)
llm = OpenAIChat(temperature=0)
# llm=OpenAI(temperature=0)
agent_executor = initialize_agent([], llm, agent="conversational-react-description", memory=memory)
agent_executor.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [ ]:
agent_executor.run(input="what's my name?")

'Your name is Bob.'